# <center>Deepseek企业级Agent项目开发实战</center>

## <center>Part 3. Ollama REST API - api/chat 接口详解 </center>

&emsp;&emsp;Ollama 服务启动后会提供一系列原生 ` REST API` 端点。通过这些`Endpoints`可以在代码环境下与`ollama`启动的大模型进行交互、管理模型和获取相关信息。其中两个`endpoint` 是最重要的，分别是：
  - <font color="red">**POST /api/generate**</font>
  - <font color="red">**POST /api/chat**</font>

&emsp;&emsp;其他端点情况：
  - POST /api/create   
  - POST /api/tags
  - POST /api/show
  - POST /api/copy
  - DELETE /api/delete
  - POST /api/pull
  - POST /api/push
  - POST /api/embed
  - GET /api/ps

---

# 1. /api/chat 接口参数概览

&emsp;&emsp;该接口使用提供的模型在聊天中生成下一条消息。与 `/api/generate` 的参数基本一致，但是在请求的参数上会根据聊天场景进行调整。主要调整的是：
- 不再使用 `prompt` 参数，而是使用 `messages` 参数。
- 新增了 `tools` 参数，用于支持工具调用。

&emsp;&emsp;其可以使用的具体参数如下所示，

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>常规参数</font></p>
<div class="center">

| 参数名       | 类型      | 描述                                                         |
| ------------ | --------- | ------------------------------------------------------------ |
| **model**    | *(必需)*  | 模型名称。                                                   |
| <font color="red">**messages**</font> | *(必需)*  | 聊天的消息，用于保持聊天记忆。                               |
| <font color="red">**tools**</font>    | *(可选)*  | JSON 中的工具列表，供模型使用（如果支持）。                 |

</div>


<p align="center"><font face="黑体" size=4>消息对象字段</font></p>
<div class="center">

| 字段名       | 描述                                                         |
| ------------ | ------------------------------------------------------------ |
| <font color="red">**role**</font>     | 消息的角色，可以是 `system`、`user`、`assistant` 或 `tool`。 |
| <font color="red">**content**</font>  | 消息的内容。                                                 |
| **images**   | *(可选)* 要在消息中包含的图像列表（适用于多模态模型，如 llava）。 |
| **tool_calls** | *(可选)* 模型希望使用的 JSON 中的工具列表。               |

</div>


<p align="center"><font face="黑体" size=4>高级参数 (可选)</font></p>
<div class="center">

| 参数名       | 描述                                                         |
| ------------ | ------------------------------------------------------------ |
| **format**   | 返回响应的格式。格式可以是 `json` 或 JSON 模式。            |
| <font color="red">**options**</font>  | 文档中列出的其他模型参数，例如 `temperature`。              |
| **stream**   | 如果为 `false`，响应将作为单个响应对象返回，而不是对象流。  |
| <font color="red">**keep_alive**</font> | 控制模型在请求后保持加载的时间（默认：5分钟）。           |

</div>

其中，Options参数说明：

| 参数名 | 描述 | 值类型 | 示例用法 |
| --------------- | ------------------------------------------------------------ | ------ | ---------------------- |
| mirostat | 启用 Mirostat 采样以控制困惑度。（默认：0，0 = 禁用，1 = Mirostat，2 = Mirostat 2.0） | int | mirostat 0 |
| mirostat_eta| 影响算法对生成文本反馈的响应速度。较低的学习率会导致调整较慢，而较高的学习率会使算法更具响应性。（默认：0.1） | float | mirostat_eta 0.1 |
| mirostat_tau| 控制输出的连贯性和多样性之间的平衡。较低的值会导致更集中和连贯的文本。（默认：5.0） | float | mirostat_tau 5.0 |
| <font color="red">num_ctx</font> | 设置用于生成下一个标记的上下文窗口大小。（默认：2048）, 影响的是模型可以一次记住的最大 token 数量。 | int | num_ctx 4096|
| repeat_last_n| 设置模型回溯的范围以防止重复。（默认：64，0 = 禁用，-1 = num_ctx） | int | repeat_last_n 64 |
| repeat_penalty| 设置惩罚重复的强度。较高的值（例如 1.5）会更强烈地惩罚重复，而较低的值（例如 0.9）会更宽松。（默认：1.1） | float | repeat_penalty 1.1 |
| <font color="red">temperature</font> | 模型的温度。增加温度会使模型的回答更具创造性。（默认：0.8） | float | temperature 0.7 |
| seed | 设置用于生成的随机数种子。将其设置为特定数字将使模型对相同提示生成相同的文本。（默认：0） | int | seed 42 |
| <font color="red">stop</font> | 设置使用的停止序列。当遇到此模式时，LLM 将停止生成文本并返回。可以通过在 modelfile 中指定多个单独的停止参数来设置多个停止模式。 | string | stop "AI assistant:" |
| <font color="red">num_predict</font> | 生成文本时要预测的最大标记数。（默认：-1，无限生成）,影响模型最大可以生成的 token 数量。 | int | num_predict 42 |
| top_k | 降低生成无意义文本的概率。较高的值（例如 100）会给出更多样化的答案，而较低的值（例如 10）会更保守。（默认：40） | int | top_k 40 |
| top_p | 与 top-k 一起工作。较高的值（例如 0.95）会导致更具多样性的文本，而较低的值（例如 0.5）会生成更集中和保守的文本。（默认：0.9） | float | top_p 0.9 |
| min_p | top_p 的替代方案，旨在确保质量和多样性之间的平衡。参数 p 表示考虑标记的最小概率，相对于最可能标记的概率。例如，p=0.05 时，最可能的标记概率为 0.9，值小于 0.045 的 logits 会被过滤掉。（默认：0.0） | float | min_p 0.05 |


# 2. requests 调用方法

&emsp;&emsp; `/api/chat` 依然还是可以`requests`库进行调用。如下所示：

In [1]:
import requests
import json

# 设置 API 端点
chat_url = "http://localhost:11434/api/chat"    # 这里需要根据实际情况进行修改

# 示例数据
chat_payload = {
    "model": "deepseek-r1:32b",   # 这里需要根据实际情况进行修改
    "messages": [
        {
            "role": "user",  # 消息角色，用户发送的消息
            "content": "请生成一个关于人工智能的简短介绍。"  # 用户的消息内容
        }
    ],
    "tools": [],  # 如果有工具可以在这里添加
    "stream": False,  # 默认使用的是True，如果设置为False，则返回的是一个完整的响应，而不是一个流式响应
}

# 调用聊天接口
response_chat = requests.post(chat_url, json=chat_payload)
if response_chat.status_code == 200:
    chat_response = response_chat.json()
    print("生成响应:", json.dumps(chat_response, ensure_ascii=False, indent=2))
else:
    print("生成请求失败:", response_chat.status_code, response_chat.text)

生成响应: {
  "model": "deepseek-r1:32b",
  "created_at": "2025-10-03T14:49:20.471145606Z",
  "message": {
    "role": "assistant",
    "content": "<think>\n好的，用户让我生成一个关于人工智能的简短介绍。首先，我需要理解什么是人工智能的基本概念。AI，也就是Artificial Intelligence，通常是指机器模拟人类智能的能力。这包括学习、推理、问题解决和自我修正等方面。\n\n接下来，我要考虑用户的使用场景。可能用户是在准备一份报告、演讲，或者只是想快速了解AI的基础知识。因此，介绍应该简洁明了，涵盖主要的应用领域，比如机器学习、自然语言处理和计算机视觉。\n\n然后是用户的身份，可能是学生、研究人员或者是对科技感兴趣的一般读者。不同的身份需要不同的深度，但在这里用户只要求简短介绍，所以重点放在关键点上，不需要太深入的技术细节。\n\n用户的真实需求是什么呢？他们可能想要一个全面但不过于复杂的概述，以便快速理解AI的基本概念和应用。此外，可能还有潜在的需求，比如了解当前的发展趋势或未来的影响，所以在介绍中可以稍微提及这些方面。\n\n结构安排的话，可以从定义开始，然后列举主要技术如机器学习、深度学习等，接着是具体的应用领域，最后提到伦理和社会影响。这样逻辑清晰，用户容易理解。\n\n语言方面要保持口语化，避免过于学术化的术语，让内容更易消化。同时，确保涵盖AI的重要性及其在现代社会中的广泛影响。\n\n最后检查一下有没有遗漏的重要点，比如监督学习、无监督学习的区别，或者其他相关技术，但为了简短，可以不展开详细讨论。主要突出关键技术和应用即可。\n</think>\n\n人工智能（Artificial Intelligence, AI）是指由人创造的机器或计算机系统所表现出的智能行为。它能够模拟人类的思维和学习能力，通过算法和大数据分析来执行任务、解决问题并做出决策。AI的核心技术包括机器学习、深度学习和自然语言处理等，广泛应用于自动驾驶、语音助手、推荐系统、医疗诊断等领域。随着技术的发展，人工智能正在深刻改变我们的生活方式和社会结构。"
  },
  "done": true,
  "done_reason":

&emsp;&emsp;返回的响应中包含以下参数，其对应的描述如下：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>响应参数</font></p>
<div class="center">

| 参数名                  | 描述                                                         |
| ----------------------- | ------------------------------------------------------------ |
| **total_duration**      | 单次响应花费的总时间                                          |
| **load_duration**       | 加载模型花费的时间                                   |
| **prompt_eval_count**   | 提示中的token数                                               |
| **prompt_eval_duration**| 评估提示所花费的时间（以纳秒为单位）                                 |
| **eval_count**          | 响应中的token数                                               |
| **eval_duration**       | 生成响应的时间（以纳秒为单位）                              |
| **context**             | 在此响应中使用的对话的编码，可以在下一个请求中发送以保持对话记忆 |
| **response**            | 空响应是流的，如果未流式传输，则将包含完整的响应             |

</div>

&emsp;&emsp;重点关注以下几个参数：

- <font color="red">**message**</font> 

&emsp;&emsp;在 `/chat` 接口中，返回的模型响应结果存放在 `message` 中， 同样对于 `DeepSeek-R1` 模型，`response` 字段中包含<think> 标签和正常文本，<think> 标签用于表示模型的思考过程或内部推理，而正常的文本则是模型生成的实际输出内容。注意：非推理类模型的返回结果中没有<think></think>标识。

In [3]:
chat_response["message"]['content']

'<think>\n好的，用户让我生成一个关于人工智能的简短介绍。首先，我需要理解什么是人工智能的基本概念。AI，也就是Artificial Intelligence，通常是指机器模拟人类智能的能力。这包括学习、推理、问题解决和自我修正等方面。\n\n接下来，我要考虑用户的使用场景。可能用户是在准备一份报告、演讲，或者只是想快速了解AI的基础知识。因此，介绍应该简洁明了，涵盖主要的应用领域，比如机器学习、自然语言处理和计算机视觉。\n\n然后是用户的身份，可能是学生、研究人员或者是对科技感兴趣的一般读者。不同的身份需要不同的深度，但在这里用户只要求简短介绍，所以重点放在关键点上，不需要太深入的技术细节。\n\n用户的真实需求是什么呢？他们可能想要一个全面但不过于复杂的概述，以便快速理解AI的基本概念和应用。此外，可能还有潜在的需求，比如了解当前的发展趋势或未来的影响，所以在介绍中可以稍微提及这些方面。\n\n结构安排的话，可以从定义开始，然后列举主要技术如机器学习、深度学习等，接着是具体的应用领域，最后提到伦理和社会影响。这样逻辑清晰，用户容易理解。\n\n语言方面要保持口语化，避免过于学术化的术语，让内容更易消化。同时，确保涵盖AI的重要性及其在现代社会中的广泛影响。\n\n最后检查一下有没有遗漏的重要点，比如监督学习、无监督学习的区别，或者其他相关技术，但为了简短，可以不展开详细讨论。主要突出关键技术和应用即可。\n</think>\n\n人工智能（Artificial Intelligence, AI）是指由人创造的机器或计算机系统所表现出的智能行为。它能够模拟人类的思维和学习能力，通过算法和大数据分析来执行任务、解决问题并做出决策。AI的核心技术包括机器学习、深度学习和自然语言处理等，广泛应用于自动驾驶、语音助手、推荐系统、医疗诊断等领域。随着技术的发展，人工智能正在深刻改变我们的生活方式和社会结构。'

&emsp;&emsp;可以通过简单的字符串操作来分离 <think> 标签中的思考内容和正常的文本内容，代码如下：

In [4]:
# 提取 <think> 标签中的内容
think_start = chat_response["message"]['content'].find("<think>")
think_end = chat_response["message"]['content'].find("</think>")

if think_start != -1 and think_end != -1:
    think_content = chat_response["message"]['content'][think_start + len("<think>"):think_end].strip()
else:
    think_content = "No think content found."

# 提取正常的文本内容
normal_content = chat_response["message"]['content'][think_end + len("</think>"):].strip()

# 打印结果
print("思考内容:\n", think_content)
print("\n正常内容:\n", normal_content)

思考内容:
 好的，用户让我生成一个关于人工智能的简短介绍。首先，我需要理解什么是人工智能的基本概念。AI，也就是Artificial Intelligence，通常是指机器模拟人类智能的能力。这包括学习、推理、问题解决和自我修正等方面。

接下来，我要考虑用户的使用场景。可能用户是在准备一份报告、演讲，或者只是想快速了解AI的基础知识。因此，介绍应该简洁明了，涵盖主要的应用领域，比如机器学习、自然语言处理和计算机视觉。

然后是用户的身份，可能是学生、研究人员或者是对科技感兴趣的一般读者。不同的身份需要不同的深度，但在这里用户只要求简短介绍，所以重点放在关键点上，不需要太深入的技术细节。

用户的真实需求是什么呢？他们可能想要一个全面但不过于复杂的概述，以便快速理解AI的基本概念和应用。此外，可能还有潜在的需求，比如了解当前的发展趋势或未来的影响，所以在介绍中可以稍微提及这些方面。

结构安排的话，可以从定义开始，然后列举主要技术如机器学习、深度学习等，接着是具体的应用领域，最后提到伦理和社会影响。这样逻辑清晰，用户容易理解。

语言方面要保持口语化，避免过于学术化的术语，让内容更易消化。同时，确保涵盖AI的重要性及其在现代社会中的广泛影响。

最后检查一下有没有遗漏的重要点，比如监督学习、无监督学习的区别，或者其他相关技术，但为了简短，可以不展开详细讨论。主要突出关键技术和应用即可。

正常内容:
 人工智能（Artificial Intelligence, AI）是指由人创造的机器或计算机系统所表现出的智能行为。它能够模拟人类的思维和学习能力，通过算法和大数据分析来执行任务、解决问题并做出决策。AI的核心技术包括机器学习、深度学习和自然语言处理等，广泛应用于自动驾驶、语音助手、推荐系统、医疗诊断等领域。随着技术的发展，人工智能正在深刻改变我们的生活方式和社会结构。


&emsp;&emsp;其他的重点参数和 `/generation` 参数使用方法也保持一致，示例代码如下：

In [7]:
import requests # type: ignore
import json

# 设置 API 端点
chat_url = "http://localhost:11434/api/chat"    # 这里需要根据实际情况进行修改

# 示例数据
chat_payload = {
    "model": "deepseek-r1:32b",   # 这里需要根据实际情况进行修改
    "messages": [
        {
            "role": "user",  # 消息角色，用户发送的消息
            "content": "请生成一个关于人工智能的简短介绍。"  # 用户的消息内容
        }
    ],
    "tools": [],  # 如果有工具可以在这里添加
    "stream": False,  # 默认使用的是True，如果设置为False，则返回的是一个完整的响应，而不是一个流式响应
    "keep_alive": "10m",   # 设置模型在请求后保持加载的时间
    "options":{
        "temperature": 0.7,   
        "num_ctx":2048,
        "num_predict": 4096,
    }
}

# 调用聊天接口
response_chat = requests.post(chat_url, json=chat_payload)
if response_chat.status_code == 200:
    chat_response = response_chat.json()
    print("生成响应:", json.dumps(chat_response, ensure_ascii=False, indent=2))
else:
    print("生成请求失败:", response_chat.status_code, response_chat.text)

生成响应: {
  "model": "deepseek-r1:32b",
  "created_at": "2025-10-03T14:54:46.089117271Z",
  "message": {
    "role": "assistant",
    "content": "<think>\n嗯，用户让我生成一个关于人工智能的简短介绍。首先，我得想想什么是AI的核心内容。可能需要包括定义、主要技术比如机器学习和深度学习，还有应用领域，比如医疗、金融、交通等等。\n\n然后，要考虑用户的使用场景。可能是学生写作业，或者是给非专业人士做讲解。所以语言要简洁明了，避免太专业的术语，但也不能太简单，得有深度。\n\n用户的身份可能是什么？也许是个学生或者刚入门的人，他们需要一个全面但简短的介绍，帮助他们快速理解AI的基本概念和重要性。\n\n深层需求可能是想了解AI的发展趋势、优势以及带来的影响。所以除了定义和技术，还要提到它如何改变生活和社会，以及未来的发展前景。\n\n结构上，先定义AI，接着讲主要技术和应用，然后讨论其优势和面临的挑战，最后展望未来。这样逻辑清晰，内容全面。\n\n字数方面，用户要求简短，但也不能太简略，大概控制在200-300字左右比较合适。\n\n现在，把这些点组织起来，确保语言流畅，信息准确。检查一下有没有遗漏的重要部分，比如伦理问题或者技术限制，这样介绍会更完整。\n</think>\n\n人工智能（Artificial Intelligence, AI）是指模拟人类智能的系统或机器，能够执行如学习、推理、问题解决和自然语言处理等任务。AI的核心技术包括机器学习、深度学习和神经网络，通过大量数据训练模型，使其具备识别模式、做出决策的能力。\n\n人工智能广泛应用于多个领域：医疗领域用于疾病诊断和药物研发；金融行业用于风险评估和交易预测；交通方面用于自动驾驶和智能调度。此外，AI还在语音助手、推荐系统和智能家居中发挥重要作用。\n\n尽管AI带来了巨大便利，但也面临伦理问题，如隐私保护和就业影响。未来，随着技术进步，人工智能将继续推动社会变革，但需在创新与责任之间找到平衡。"
  },
  "done": true,
  "done_reason": "stop",
  "total_duration": 19114086413,
 

&emsp;&emsp;流式输出代码也要针对`/chat`接口的返回响应格式做略微的修改：

In [9]:
import requests  # type: ignore
import json

# 设置 API 端点（使用 chat 接口）
chat_url = "http://localhost:11434/api/chat"

# 示例数据（chat 接口格式）
chat_payload = {
    "model": "deepseek-r1:32b",  # 使用已下载的模型
    "messages": [  # chat 接口使用 messages 数组
        {
            "role": "user",
            "content": "请生成一个关于人工智能的简短介绍。"
        }
    ],
    "stream": True,  # 启用流式输出
    "options": {
        "temperature": 0.6,
        "num_predict": 200
    }
}

print("开始对话...\n")

# 调用 chat 接口
try:
    with requests.post(chat_url, json=chat_payload, stream=True, timeout=60) as response_chat:
        if response_chat.status_code == 200:
            # 逐行读取流式响应
            for line in response_chat.iter_lines():
                if line:  # 确保行不为空
                    try:
                        # 解析 JSON 响应
                        chat_response = json.loads(line)
                        
                        # chat 接口返回 message.content
                        if "message" in chat_response and "content" in chat_response["message"]:
                            print(chat_response["message"]["content"], end='', flush=True)
                        
                        # 检查是否完成
                        if chat_response.get("done", False):
                            print("\n\n✅ 对话完成")
                            # 打印统计信息
                            if "total_duration" in chat_response:
                                duration = chat_response["total_duration"] / 1e9
                                print(f"总耗时: {duration:.2f} 秒")
                            if "eval_count" in chat_response:
                                tokens = chat_response["eval_count"]
                                print(f"生成 tokens: {tokens}")
                            break
                    except json.JSONDecodeError as e:
                        print(f"\n解析错误: {e}")
                        continue
        else:
            print(f"❌ 对话请求失败: {response_chat.status_code}")
            print(f"错误信息: {response_chat.text}")
except requests.exceptions.RequestException as e:
    print(f"❌ 请求异常: {e}")
    print("提示: 请确保 Ollama 服务正在运行 (ollama serve)")



开始对话...

<think>
嗯，用户让我生成一个关于人工智能的简短介绍。我得先想想怎么组织内容。

首先，定义AI是什么。可能要提到它模拟人类智能的技术，包括学习、推理和自我修正这些方面。然后，列举一些常见的技术应用，比如机器学习、深度学习和自然语言处理。

接下来，说明AI的应用领域，比如医疗、金融、制造等等，这样用户能明白它的广泛影响。还要提到它的优势，比如高效性和准确性，但也要平衡一下，讲讲伦理和社会问题，比如隐私和就业影响。

最后，强调AI作为双刃剑，需要负责任的开发来最大化利益，同时减少风险。这样内容就比较全面了。

语言要简洁明了，避免太专业的术语，让不同背景的人都能理解。
</think>

人工智能（Artificial Intelligence, AI）是指模拟人类智能的技术和系统的科学。它通过机器学习、深度学习和自然语言处理等技术，使计算机能够执行

✅ 对话完成
总耗时: 6.20 秒
生成 tokens: 200
